In [3]:
import pandas, matplotlib.pyplot
import seaborn
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
from datetime import datetime
import numpy as np
import missingno as msno
from sklearn.decomposition import PCA

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression





In [10]:
storms = pd.read_csv('./data/storms.csv', index_col=0)



In [11]:
storms.head()

,name,year,month,day,hour,lat,long,status,category,wind,pressure,tropicalstorm_force_diameter,hurricane_force_diameter
1,Amy,1975,6,27,0,27.5,-79.0,tropical depression,NaN,25,1013,NaN,NaN
2,Amy,1975,6,27,6,28.5,-79.0,tropical depression,NaN,25,1013,NaN,NaN
3,Amy,1975,6,27,12,29.5,-79.0,tropical depression,NaN,25,1013,NaN,NaN
4,Amy,1975,6,27,18,30.5,-79.0,tropical depression,NaN,25,1013,NaN,NaN
5,Amy,1975,6,28,0,31.5,-78.8,tropical depression,NaN,25,1012,NaN,NaN


In [12]:
storms.dtypes

name                             object
year                              int64
month                             int64
day                               int64
hour                              int64
lat                             float64
long                            float64
status                           object
category                        float64
wind                              int64
pressure                          int64
tropicalstorm_force_diameter    float64
hurricane_force_diameter        float64
dtype: object

In [21]:
storms.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19066 entries, 1 to 19066
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   name                          19066 non-null  object 
 1   year                          19066 non-null  int64  
 2   month                         19066 non-null  int64  
 3   day                           19066 non-null  int64  
 4   hour                          19066 non-null  int64  
 5   lat                           19066 non-null  float64
 6   long                          19066 non-null  float64
 7   status                        19066 non-null  object 
 8   category                      4684 non-null   float64
 9   wind                          19066 non-null  int64  
 10  pressure                      19066 non-null  int64  
 11  tropicalstorm_force_diameter  9554 non-null   float64
 12  hurricane_force_diameter      9554 non-null   float64
dtypes: flo

In [27]:
storms[['category','tropicalstorm_force_diameter']].isnull().value_counts()

category  tropicalstorm_force_diameter
True      False                           7503
          True                            6879
False     True                            2633
          False                           2051
Name: count, dtype: int64

In [9]:
storms.duplicated().sum()

0

In [13]:
#Assesing percentage of missing values by features

missing = pd.concat([storms.isnull().sum(), 100*storms.isnull().sum()/storms.shape[0]], axis=1)
missing.columns = ['Count', '%']
missing.sort_values(by='%', ascending=False, inplace=True)

missing[missing['%']>10]

,Count,%
category,14382,75.432707
tropicalstorm_force_diameter,9512,49.889856
hurricane_force_diameter,9512,49.889856


In [20]:
storms.describe()

,year,month,day,hour,lat,long,category,wind,pressure,tropicalstorm_force_diameter,hurricane_force_diameter
count,19066.000000,19066.000000,19066.000000,19066.000000,19066.000000,19066.000000,4684.000000,19066.000000,19066.000000,9554.000000,9554.000000
mean,2002.278926,8.698626,15.781968,9.094042,26.994252,-61.524300,1.898377,50.017413,993.554390,146.309399,14.811074
std,12.556517,1.352956,8.878563,6.733683,10.414302,21.062519,1.150590,25.501030,18.737342,156.006026,34.083358
min,1975.000000,1.000000,1.000000,0.000000,7.000000,-109.300000,1.000000,10.000000,882.000000,0.000000,0.000000
25%,1993.000000,8.000000,8.000000,5.000000,18.400000,-78.700000,1.000000,30.000000,987.000000,0.000000,0.000000
50%,2004.000000,9.000000,16.000000,12.000000,26.600000,-62.250000,1.000000,45.000000,1000.000000,110.000000,0.000000
75%,2012.000000,9.000000,24.000000,18.000000,33.700000,-45.600000,3.000000,65.000000,1007.000000,220.000000,0.000000
max,2021.000000,12.000000,31.000000,23.000000,70.700000,13.500000,5.000000,165.000000,1024.000000,1440.000000,300.000000
